In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
from utils import *
import numpy as np
torch.manual_seed(0)

In [2]:
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,
                                         shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:

import torch.nn as nn

model = Resnet18_2exits_Pyramidpool().cuda()


In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.6, weight_decay=5e-4)
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.5)
#model, optimizer = amp.initialize(model, optimizer, opt_level="O2")
#lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
#                                                              T_0=int(len(trainset)/64*2),
     #                                                         T_mult=1)

In [5]:
%%time
epochs= 150
coefficient = [0.3,0.7]
loss = []
for epoch in range(epochs):
    if epoch%25 == 0 and epoch!=0:
        my_lr_scheduler.step()
        print(optimizer.param_groups[0]['lr'])
        
        # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        exit1,exit2, = model(inputs)
        loss1 = criterion(exit1, labels)
        loss2 = criterion(exit2, labels)
        #loss3 = criterion(exit3, labels)
        total_loss = coefficient[0]*loss1+coefficient[1]*loss2
       # with amp.scale_loss(loss, optimizer) as scaled_loss:
        #    scaled_loss.backward()
        total_loss.backward()
        optimizer.step()
        #lr_scheduler.step()
        # print statistics
        running_loss += total_loss.item()
        
        
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss /99))
            loss.append(running_loss)
            running_loss = 0.0

    torch.save(model,"Pyramid_2_exits.pth")
print('Finished Training')

[1,   100] loss: 1.49971
[2,   100] loss: 0.92502
[3,   100] loss: 0.80400
[4,   100] loss: 0.72906
[5,   100] loss: 0.68492
[6,   100] loss: 0.62914
[7,   100] loss: 0.60004
[8,   100] loss: 0.58075
[9,   100] loss: 0.55643
[10,   100] loss: 0.53000
[11,   100] loss: 0.51498
[12,   100] loss: 0.49274
[13,   100] loss: 0.49444
[14,   100] loss: 0.48575
[15,   100] loss: 0.45953
[16,   100] loss: 0.44847
[17,   100] loss: 0.44738
[18,   100] loss: 0.43706
[19,   100] loss: 0.42498
[20,   100] loss: 0.42027
[21,   100] loss: 0.41519
[22,   100] loss: 0.40127
[23,   100] loss: 0.40035
[24,   100] loss: 0.39643
[25,   100] loss: 0.38635
0.05
[26,   100] loss: 0.30852
[27,   100] loss: 0.27866
[28,   100] loss: 0.26337
[29,   100] loss: 0.26368
[30,   100] loss: 0.26128
[31,   100] loss: 0.25402
[32,   100] loss: 0.24306
[33,   100] loss: 0.25393
[34,   100] loss: 0.24399
[35,   100] loss: 0.24395
[36,   100] loss: 0.23790
[37,   100] loss: 0.23172
[38,   100] loss: 0.23258
[39,   100] loss